In [1]:
import copy
import heapq
import itertools as its
import math
import os
import pathlib
import re
import string
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec18.txt'

In [2]:
WALL = '#'
START = '@'
EMPTY = '.'

def read_maze(text: str):
    maze = {}
    start_pos = None
    doors = {}
    keys = {}
    for y, line in enumerate(text.rstrip().split('\n')):
        for x, c in enumerate(line.rstrip()):
            pos = (x, y)
            if c != WALL:
                maze[pos] = c
            if c == START:
                start_pos = pos
            if c in string.ascii_uppercase:
                doors[pos] = c
            if c in string.ascii_lowercase:
                keys[pos] = c
    return maze, start_pos, doors, keys

In [3]:
maze, start_pos, doors, keys = read_maze(INPUT_PATH.read_text())

In [4]:
test_maze_input = """\
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################"""

test_input_2 = """\
#########
#b.A.@.a#
#########"""

test_input_3 = """\
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""

test_input_4 = """\
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################"""

In [5]:
def print_maze(maze, start_pos, doors, keys):
    min_x = min(key[0] for key in maze)
    max_x = max(key[0] for key in maze)
    min_y = min(key[1] for key in maze)
    max_y = max(key[1] for key in maze)
    
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            pos = (x, y)
            if pos in maze:
                if pos in doors:
                    print(doors[pos], end='')
                elif pos in keys:
                    print(keys[pos], end='')
                elif pos == start_pos:
                    print('@', end='')
                else:
                    print('.', end='')
            else:
                print('#', end='')
        print()


In [6]:
# Slow search method

def search(maze, start_pos, doors, keys):
    queue = deque([(start_pos, frozenset(''), 0)])
    seen = set([(start_pos, frozenset(''))])
    
    def append_to_queue(pos, keys_i_have, path_length):
        propose = (pos, keys_i_have)
        if propose not in seen:
            seen.add(propose)
            queue.append(propose + (path_length,))

    while queue:
        pos, keys_i_have, path_length = queue.popleft()
        for to_pos in util.four_ways(*pos):
            if to_pos not in maze:
                continue

            if to_pos in doors:
                # If I'm a door
                if doors[to_pos].lower() in keys_i_have:
                    # And I have the key, step through the door. Else nope out
                    append_to_queue(to_pos, keys_i_have, path_length + 1)
            elif to_pos in keys:
                # If I'm a key
                key = keys[to_pos]
                local_keys_i_have = keys_i_have | frozenset(key)  # Collect the key
                if len(local_keys_i_have) == len(keys):
                    # Done!
                    return path_length + 1

                # Step onto the square
                append_to_queue(to_pos, local_keys_i_have, path_length + 1)
            else:
                # Just a regular square
                append_to_queue(to_pos, keys_i_have, path_length + 1)
                

# Faster search method (compresses to a key you can get; doesn't bother with queue for empty steps)
def keys_i_can_get(maze, start_pos, doors, keys_i_have):
    queue = deque([(start_pos, 0)])
    seen = set([start_pos])
    
    while queue:
        pos, dist = queue.popleft()
        for to_pos in util.four_ways(*pos):
            if to_pos not in maze or to_pos in seen:
                continue
            
            if to_pos in doors and doors[to_pos].lower() not in keys_i_have:
                continue

            if to_pos in keys and keys[to_pos] not in keys_i_have:
                yield to_pos, keys[to_pos], dist + 1
                continue
            
            queue.append((to_pos, dist + 1))
            seen.add(to_pos)
                    

def search_fast(maze, start_pos, doors, keys):
    # Distance (heap invariant), starting position (array), keys i have
    heapqueue = [(0, start_pos, frozenset())]
    seen = [set() for _ in range(len(start_pos))]
    while heapqueue:
        dist, cur_pos, keys_i_have = heapq.heappop(heapqueue)
        for i, pos in enumerate(cur_pos):
            if len(keys_i_have) == len(keys):
                return dist
        
            if (pos, keys_i_have) in seen[i]:
                continue
            
            seen[i].add((pos, keys_i_have))
            
            for new_small_pos, new_key, new_dist in keys_i_can_get(maze, pos, doors, keys_i_have):
                new_pos = cur_pos[:i] + (new_small_pos,) + cur_pos[i+1:]
                heapq.heappush(heapqueue, (dist + new_dist, new_pos, keys_i_have | frozenset([new_key])))

In [7]:
maze, start_pos, doors, keys = read_maze(test_input_2)
print_maze(maze, start_pos, doors, keys)
assert search(maze, start_pos, doors, keys) == 8

b.A.@.a


In [8]:
maze, start_pos, doors, keys = read_maze(test_input_3)
print_maze(maze, start_pos, doors, keys)
assert search(maze, start_pos, doors, keys) == 86
assert search_fast(maze, (start_pos,), doors, keys) == 86

f.D.E.e.C.b.A.@.a.B.c.
#####################.
d.....................


In [9]:
maze, start_pos, doors, keys = read_maze(test_maze_input)
print_maze(maze, start_pos, doors, keys)
assert search(maze, start_pos, doors, keys) == 132
assert search_fast(maze, (start_pos,), doors, keys) == 132

...............b.C.D.f
.#####################
.....@.a.B.c.d.A.e.F.g


In [10]:
maze, start_pos, doors, keys = read_maze(test_input_4)
print_maze(maze, start_pos, doors, keys)
assert search(maze, start_pos, doors, keys) == 136
assert search_fast(maze, (start_pos,), doors, keys) == 136

i.G..c...e..H.p
#######.#######
j.A..b...f..D.o
#######@#######
k.E..a...g..B.n
#######.#######
l.F..d...h..C.m


In [11]:
maze, start_pos, doors, keys = read_maze(INPUT_PATH.read_text())
print(f'The answer to part 1 is {search_fast(maze, (start_pos,), doors, keys)}')

The answer to part 1 is 4868


In [12]:
# For part 2 our functions are a bit more complicated

def convert(maze, start_pos):
    """ Convert the maze to the part 2 format """
    maze = copy.copy(maze)
    del maze[start_pos]
    for pos in util.four_ways(*start_pos):
        del maze[pos]
    start_poses = []
    for i in [-1, 1]:
        for j in [-1, 1]:
            start_poses.append((start_pos[0] + i, start_pos[1] + j))
    start_poses = tuple(start_poses)
    return maze, start_poses

In [13]:
test_again = """\
#######
#a.#Cd#
##...##
##.@.##
##...##
#cB#Ab#
#######"""

test_again2 = """\
###############
#d.ABC.#.....a#
######...######
######.@.######
######...######
#b.....#.....c#
###############"""

In [14]:
maze, start_pos, doors, keys = read_maze(test_again)
maze, start_poses = convert(maze, start_pos)
assert search_fast(maze, start_poses, doors, keys) == 8

In [15]:
maze, start_pos, doors, keys = read_maze(test_again2)
maze, start_poses = convert(maze, start_pos)
assert search_fast(maze, start_poses, doors, keys) == 24

In [16]:
maze, start_pos, doors, keys = read_maze(INPUT_PATH.read_text())
maze, start_poses = convert(maze, start_pos)
print(f'The answer to part 2 is {search_fast(maze, start_poses, doors, keys)}')

The answer to part 2 is 1984
